In [ ]:
import logging

import networkx as nx

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import causalnex
from causalnex.plots import plot_structure
from causalnex.structure import StructureModel

plot_structure(StructureModel())
print(causalnex.__version__)

In [ ]:
from autorocks.data.loader.all_models_result_aggregator import (
    create_all_models_comparison_dataset,
)
from autorocks.envs.synthetic.funcs.ackley import Ackley6DParametersSpace
from autorocks.dir_struct import LocalResultDir


exp_dir = LocalResultDir / f"synthetic/target/Akcley/6_params/100_iter"
model_comparison_data = create_all_models_comparison_dataset(exp_dir)

param_space = Ackley6DParametersSpace()
param_names = set([p.name for p in param_space.parameters()])
main_targets = {"target"}

In [ ]:
import pandas as pd

random_dataset = model_comparison_data.filter_for_specific_models({"Random"})
uuid_cols = ["model", "iteration", "step"]
all_exp_dfs = pd.merge(random_dataset.parameters, random_dataset.system_performance, on=uuid_cols).drop(
    columns=uuid_cols
)
# params_and_objectives =  model_comparison_data

In [ ]:
intermediate_metrics = set(all_exp_dfs.columns) - param_names - main_targets

intermediate_metrics = {"structure.lhd", "structure.rhd"}

# Preprocessing steps


In [ ]:
from autorocks.optimizer.bograph.bograph_dao import BoGraphDataPandas

params: pd.DataFrame = all_exp_dfs[param_names]
objs: pd.DataFrame = all_exp_dfs[main_targets]
intermediate: pd.DataFrame = all_exp_dfs[intermediate_metrics]

bograph_data = BoGraphDataPandas(params=params, objs=objs, intermediate=intermediate)

In [ ]:
from autorocks.optimizer.bograph.preprocessor.standardizer import (
    MetricsStandardizerProcessor,
)
from autorocks.optimizer.bograph.dag_preprocessor import PreprocessingPipeline

from autorocks.optimizer.bograph.preprocessor.normalizer import ParamNormalizerProcessor

preprocessing_pipeline = PreprocessingPipeline(
    [
        ParamNormalizerProcessor(param_space.bounds(True).T),
        MetricsStandardizerProcessor(),
    ]
)
processed_data = preprocessing_pipeline.fit_transform(bograph_data.copy())

# Structure learning

In [ ]:
from botorch.utils import standardize
from causalnex.structure.pytorch import from_pandas
import torch

torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.set_default_dtype(torch.float32)


sm = from_pandas(
    processed_data.to_combi_pandas(),
    # w_threshold=0.8,
    tabu_parent_nodes=main_targets,
    tabu_child_nodes=param_names,
    tabu_edges=[["structure.lhd", "structure.rhd"], ["structure.rhd", "structure.lhd"]],
    # hidden_layer_units=[128],
    # use_bias=True,
    # ridge_beta=0.1
)
print("Done")

In [ ]:
from autorocks.viz.causal_util import plot_struct_customized
from IPython.display import Image

smaller_sm = sm.copy()
smaller_sm.remove_edges_below_threshold(0.01)
viz = plot_struct_customized(
    smaller_sm.get_target_subgraph("target"),
    graph_name="Ackley Structure",
    param_nodes=param_names,
    sink_nodes=main_targets,
)
Image(viz.draw(format="png"))

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

In [ ]:
from castle.algorithms import MCSL, GOLEM

model = MCSL(device_type="gpu", max_iter=1)  # num_iter=10000)

combi_pandas = processed_data.to_combi_pandas()
model.learn(combi_pandas.values, columns=combi_pandas.columns)

In [ ]:
from castle.common import GraphDAG

true_dag = nx.from_pandas_adjacency(
    pd.DataFrame(
        model.causal_matrix,
        index=model.causal_matrix.columns,
        columns=model.causal_matrix.columns,
    )
)
true_dag.remove_edges_from(list(true_dag.edges))

true_dag.add_edges_from([(p, f"structure.cos({p})") for p in param_space.keys()])
true_dag.add_edges_from([(p, f"structure.pow({p})") for p in param_space.keys()])
true_dag.add_edges_from([(f"structure.pow({p})", "structure.lhd") for p in param_space.keys()])
true_dag.add_edges_from([(f"structure.cos({p})", "structure.rhd") for p in param_space.keys()])
true_dag.add_edges_from([("structure.lhd", "target"), ["structure.rhd", "target"]])

from castle.common import Tensor

true_dag_np = Tensor(
    nx.to_numpy_matrix(true_dag),
    index=model.causal_matrix.columns,
    columns=model.causal_matrix.columns,
)

GraphDAG(model.causal_matrix, show=True, true_dag=true_dag_np)

In [ ]:
nx.from_pandas_adjacency(
    pd.DataFrame(
        model.causal_matrix,
        index=model.causal_matrix.columns,
        columns=model.causal_matrix.columns,
    )
).edges